In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

## Aggregieren der Reden pro Partei

In [ ]:
# Daten einlesen
reden_df = pd.read_csv("reden_relevant.csv")
artikel_df = pd.read_csv("artikel_relevant.csv")
print(len(reden_df))
print(len(artikel_df))

In [ ]:
# PDS mit Linke ersetzen
reden_df['partei'] = reden_df['partei'].replace({'PDS': 'LINKE'})

### TF-IDF-Werte

In [ ]:
# TF-IDF-Berechnung für Reden
vectorizer_reden = TfidfVectorizer()
tfidf_reden = vectorizer_reden.fit_transform(reden_df['text_preprocessed_specific'])

In [ ]:
# TF-IDF-Berechnung für Artikel
vectorizer_artikel = TfidfVectorizer()
tfidf_artikel = vectorizer_artikel.fit_transform(artikel_df['text_preprocessed_specific'])

Aggregieren der TF-IDF-Werte

In [ ]:
# TF-IDF-Werte mit Parteien verknüpfen (Reden)
reden_tfidf_df = pd.DataFrame(tfidf_reden.toarray(), columns=vectorizer_reden.get_feature_names_out())
reden_tfidf_df['partei'] = reden_df['partei']

# TF-IDF-Werte nach Partei aggregieren (Reden)
partei_tfidf = reden_tfidf_df.groupby('partei').mean()

In [ ]:
# TF-IDF-Werte mit den Jahresinformationen verknüpfen (Artikel)
tfidf_artikel_df = pd.DataFrame(tfidf_artikel.toarray(), columns=vectorizer_artikel.get_feature_names_out()) # Convert to DataFrame
tfidf_artikel_df['Jahr'] = artikel_df['datum_jahr']  # Add the 'Jahr' column

# TF-IDF-Werte nach Jahr aggregieren (mittlere Werte) (Artikel)
artikel_tfidf = tfidf_artikel_df.groupby('Jahr').mean() # groupby using the DataFrame

In [ ]:
# Optional: Speichern der aggregierten Werte
partei_tfidf.to_csv("partei_tfidf.csv", index=True)
artikel_tfidf.to_csv("artikel_tfidf.csv", index=False)

### Kosinus-Ähnlichkeit

In [ ]:
# Aggregierte TF-IDF-Werte der Parteien laden (aus vorherigem Schritt)
partei_tfidf = pd.read_csv("partei_tfidf.csv", index_col=0)

In [ ]:
# Schnittmenge der Vokabulare finden
gemeinsame_woerter = list(set(partei_tfidf.columns).intersection(artikel_tfidf.columns))

# Matritzen auf gemeinsame Wörter reduzieren
partei_tfidf_reduced = partei_tfidf[gemeinsame_woerter]
artikel_tfidf_reduced = artikel_tfidf[gemeinsame_woerter]

In [ ]:
# Kosinus-Ähnlichkeit zwischen den jährlichen Artikel-TF-IDF-Werten und den Partei-TF-IDF-Werten
yearly_similarity_matrix = cosine_similarity(artikel_tfidf_reduced.values, partei_tfidf_reduced.values)

# Ähnlichkeit in einen DataFrame umwandeln
yearly_similarity_df = pd.DataFrame(yearly_similarity_matrix, columns=partei_tfidf.index, index=artikel_tfidf.index)

# Ergebnis anzeigen
print(yearly_similarity_df.head())

In [ ]:
yearly_similarity_df.to_csv("ähnlichkeit_parteien_relevant.csv", index=True, sep=';')

In [ ]:
# Partei mit der höchsten Ähnlichkeit pro Jahr
yearly_results = yearly_similarity_df.idxmax(axis=1)
yearly_similarity_with_scores = pd.DataFrame({
    'Jahr': yearly_similarity_df.index,
    'ähnlichste_partei': yearly_results,
    'höchste_ähnlichkeit': yearly_similarity_df.max(axis=1)
})

# Ergebnisse anzeigen
print(yearly_similarity_with_scores)

### Graphische Darstellung

In [ ]:
# Sicherstellen, dass der Index ein Datum ist
yearly_similarity_df.index = pd.to_datetime(yearly_similarity_df.index, format='%Y')

# Daten bis zum 01.10.2021 filtern
filtered_df = yearly_similarity_df[yearly_similarity_df.index <= '2021-10-01']

# Parteien Farben zuordnen
farben = {
    'CDU/CSU': 'black',
    'SPD': 'red',
    'FDP': 'gold',
    'LINKE': 'deeppink',
    'GRÜNE': 'limegreen',
    'AfD': 'dodgerblue',
    'Cabinet': 'midnightblue',
    'fraktionslos': 'gray'
}

# Plot erstellen mit individuellen Farben
ax = filtered_df.plot(kind='line', figsize=(10, 6), title='Ähnlichkeit der Artikel mit Parteien', color=[farben[col] for col in filtered_df.columns])

# Achsenbeschriftung
plt.ylabel('Kosinus-Ähnlichkeit')

# Einfügen der Legende
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=4, frameon=False)

# Speichern der Grafik
plt.savefig('ähnlichkeit_parteien_artikel.png', format='png', dpi=600)

# Plot anzeigen
plt.show()

## Agreggieren der Reden pro Partei und Jahr

In [ ]:
# Daten einlesen
reden_df = pd.read_csv("reden_relevant.csv")
artikel_df = pd.read_csv("artikel_relevant.csv")
print(len(reden_df))
print(len(artikel_df))

In [ ]:
# PDS mit Linke ersetzen
reden_df['partei'] = reden_df['partei'].replace({'PDS': 'LINKE'})

In [ ]:
# artikel_tfidf = pd.read_csv("artikel_tfidf.csv")

In [ ]:
print(len(artikel_tfidf))

### TF-IDF-Werte

In [ ]:
# TF-IDF-Berechnung für Artikel
vectorizer_artikel = TfidfVectorizer()
artikel_tfidf = vectorizer_artikel.fit_transform(artikel_df['text_preprocessed_specific'])

In [ ]:
# TF-IDF-Werte mit den Jahresinformationen verknüpfen (Artikel)
artikel_tfidf_df = pd.DataFrame(artikel_tfidf.toarray(), columns=vectorizer_artikel.get_feature_names_out()) # Convert to DataFrame
artikel_tfidf_df['Jahr'] = artikel_df['datum_jahr']  # Add the 'Jahr' column

# TF-IDF-Werte nach Jahr aggregieren (mittlere Werte) (Artikel)
aggregated_artikel_tfidf = artikel_tfidf_df.groupby('Jahr').mean() # groupby using the DataFrame

In [ ]:
aggregated_tfidf_artikel.head()

In [ ]:
# Gruppieren der Reden nach Partei und Jahr (Reden)
reden_df['datum_jahr'] = reden_df['datum_jahr'].astype(str)  # Sicherstellen, dass datum_jahr als String vorliegt
gruppen = reden_df.groupby(['datum_jahr', 'partei'])['text_preprocessed_specific'].apply(lambda x: ' '.join(x))

# TF-IDF-Vektorisierung
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(gruppen)

# DataFrame für TF-IDF-Werte erstellen
tfidf_reden_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out(), index=gruppen.index)

# Index aufspalten in datum_jahr und Partei
tfidf_reden_df.index = pd.MultiIndex.from_tuples(tfidf_reden_df.index, names=['datum_jahr', 'Partei'])

# Ergebnis anzeigen
# print(tfidf_reden_df.head())

In [ ]:
# Schnittmenge der Vokabulare finden
gemeinsame_woerter = list(set(tfidf_reden_df.columns).intersection(aggregated_tfidf_artikel.columns))

# Matritzen auf gemeinsame Wörter reduzieren
partei_tfidf_reduced = tfidf_reden_df[gemeinsame_woerter]
artikel_tfidf_reduced = aggregated_tfidf_artikel[gemeinsame_woerter]

# Reset the index of partei_tfidf_reduced to access 'datum_jahr' and 'Partei' as columns
partei_tfidf_reduced = partei_tfidf_reduced.reset_index()

# Die Spalte 'datum_jahr' auffüllen (jedes Jahr auf nachfolgende Zeilen übertragen)
partei_tfidf_reduced['datum_jahr'] = partei_tfidf_reduced['datum_jahr'].fillna(method='ffill')

# Optional: Umbenennen der Spalte 'datum_jahr' für Klarheit
partei_tfidf_reduced.rename(columns={'datum_jahr': 'Jahr'}, inplace=True)

In [ ]:
# Konvertiere den Index von artikel_tfidf_reduced in eine Spalte
artikel_tfidf_reduced.reset_index(inplace=True)

In [ ]:
# Beispielhafte Ausgabe an Wörtern und ihren TF-IDF-Werten (Artikel)
artikel_tfidf_reduced.head()

In [ ]:
# Optional: Speichern der Wörter
artikel_tfidf_reduced.to_csv("artikel_tfidf.csv", index=False)

In [ ]:
# Beispielhafte Ausgabe an Wörtern und ihren TF-IDF-Werten (Reden)
partei_tfidf_reduced.head()

In [ ]:
# Optional: Speichern der Wörter
partei_tfidf_reduced.to_csv("partei_tfidf.csv", index=False)

### Kosinus-Ähnlichkeit

In [ ]:
# Sicherstellen, dass 'Jahr' als Integer vorliegt
partei_tfidf_reduced['Jahr'] = partei_tfidf_reduced['Jahr'].astype(int)
artikel_tfidf_reduced['Jahr'] = artikel_tfidf_reduced['Jahr'].astype(int)

# Ergebnisse speichern
similarities = []

# Gemeinsame Jahre ermitteln
gemeinsame_jahre = set(artikel_tfidf_reduced['Jahr']).intersection(set(partei_tfidf_reduced['Jahr']))
print("Gemeinsame Jahre:", gemeinsame_jahre)

# Nur über gemeinsame Jahre iterieren
for jahr in gemeinsame_jahre:
    # Artikel-TF-IDF-Vektor für das aktuelle Jahr
    artikel_vec = artikel_tfidf_reduced[artikel_tfidf_reduced['Jahr'] == jahr].iloc[:, 1:].values  # Ohne Jahr-Spalte

    # Partei-TF-IDF-Vektoren und Labels für das aktuelle Jahr
    partei_data = partei_tfidf_reduced[partei_tfidf_reduced['Jahr'] == jahr]
    print(f"Jahr {jahr}: Partei-Daten gefunden:", len(partei_data))  # Debug-Ausgabe
    partei_vecs = partei_data.iloc[:, 2:].values  # Ohne Jahr und Partei
    partei_labels = partei_data['Partei'].values

    # Check if either partei_vecs or artikel_vec are empty and if so, skip to next year
    if len(partei_vecs) == 0 or len(artikel_vec) == 0:
        print(f"Skipping year {jahr} due to missing data (partei_vecs: {len(partei_vecs)}, artikel_vec: {len(artikel_vec)})")
        continue

    # Berechnung der Kosinus-Ähnlichkeiten (nur wenn Daten vorhanden sind)
    jahr_similarities = cosine_similarity(artikel_vec, partei_vecs).flatten()

    # Partei mit der höchsten Ähnlichkeit finden
    most_similar_idx = jahr_similarities.argmax()
    most_similar_party = partei_labels[most_similar_idx]
    highest_similarity = jahr_similarities[most_similar_idx]

    # Ergebnis speichern
    similarities.append({
        'Jahr': jahr,
        'Partei': most_similar_party,
        'Ähnlichkeit': highest_similarity
    })

# Ergebnisse in DataFrame umwandeln
similarity_df = pd.DataFrame(similarities)

# Überprüfen, on similarity_df leer ist
if similarity_df.empty:
    print("No similarities were calculated. Please check your data for overlapping years and parties.")
else:
    # Ergebnisse sortieren und anzeigen
    similarity_df_sorted = similarity_df.sort_values(by='Jahr')
    print(similarity_df_sorted)


Kosinus-Ähnlichkeit für jedes Jahr und jede Partei bestimmen und speichern.

In [ ]:
# Sicherstellen, dass 'Jahr' als Integer vorliegt
partei_tfidf_reduced['Jahr'] = partei_tfidf_reduced['Jahr'].astype(int)
artikel_tfidf_reduced['Jahr'] = artikel_tfidf_reduced['Jahr'].astype(int)

# Ergebnisse speichern
similarities = []

# Gemeinsame Jahre ermitteln
gemeinsame_jahre = set(artikel_tfidf_reduced['Jahr']).intersection(set(partei_tfidf_reduced['Jahr']))

# Nur über gemeinsame Jahre iterieren
for jahr in gemeinsame_jahre:
    # Artikel-TF-IDF-Vektor für das aktuelle Jahr
    artikel_vec = artikel_tfidf_reduced[artikel_tfidf_reduced['Jahr'] == jahr].iloc[:, 1:].values  # Ohne Jahr-Spalte

    # Partei-TF-IDF-Vektoren und Labels für das aktuelle Jahr
    partei_data = partei_tfidf_reduced[partei_tfidf_reduced['Jahr'] == jahr]
    partei_vecs = partei_data.iloc[:, 2:].values  # Ohne Jahr und Partei
    partei_labels = partei_data['Partei'].values

    # Überprüfen, ob Jahr leer ist und überspringen
    if len(partei_vecs) == 0 or len(artikel_vec) == 0:
        print(f"Überspringe {jahr} aufgrund fehlender Daten (partei_vecs: {len(partei_vecs)}, artikel_vec: {len(artikel_vec)})")
        continue

    # Berechnung der Kosinus-Ähnlichkeiten
    jahr_similarities = cosine_similarity(artikel_vec, partei_vecs).flatten()

    # Speichere die Ähnlichkeit für jede Partei
    for partei_idx, partei in enumerate(partei_labels):
        similarities.append({
            'Jahr': jahr,
            'Partei': partei,
            'Ähnlichkeit': jahr_similarities[partei_idx]
        })

# Ergebnisse in DataFrame umwandeln
similarity_df = pd.DataFrame(similarities)

In [ ]:
# DataFrame vorbereiten
similarity_pivot = similarity_df.pivot(index='Jahr', columns='Partei', values='Ähnlichkeit')

In [ ]:
print(similarity_pivot)

### Grpahische Darstellung

In [ ]:
# Farben für jede Partei festlegen
farben = {
    'CDU/CSU': 'black',
    'SPD': 'red',
    'FDP': 'gold',
    'LINKE': 'deeppink',
    'GRÜNE': 'limegreen',
    'AfD': 'dodgerblue',
    'Cabinet': 'midnightblue',
    'fraktionslos': 'gray'
}

# Plot mit individuellen Farben erstellen
plt.figure(figsize=(12, 6))
for party in similarity_pivot.columns:
    plt.plot(similarity_pivot.index, similarity_pivot[party], label=party, color=farben.get(party, 'black'))

# Achsenbeschriftung
plt.ylabel('Kosinus-Ähnlichkeit')
plt.xlabel('Jahr')

# Einfügen der Legende
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=4, frameon=False)

plt.savefig('ähnlichkeit_parteien_artikel_relevant.png', format='png', dpi=600)

plt.show()

## Aussagekräftigste Wörter pro Jahr

### In den Artikeln

In [ ]:
# DataFrame zur Speicherung der Top-Wörter vorbereiten
top_words_articles = {}

# Iterieren über die Jahre in artikel_tfidf_reduced
for jahr in artikel_tfidf_reduced['Jahr'].unique():
    # Filtern der TF-IDF-Werte für das aktuelle Jahr
    jahr_data = artikel_tfidf_reduced[artikel_tfidf_reduced['Jahr'] == jahr].iloc[:, 1:]  # Ohne Jahr-Spalte

    # Umwandlen der TF-IDF-Werte und Wörter in eine sortierbare Struktur
    tfidf_values = jahr_data.values.flatten()
    words = jahr_data.columns
    top_indices = tfidf_values.argsort()[::-1][:10]  # Indizes der Top 10 höchsten Werte

    # Speichern der Wörter für das aktuelle Jahr
    top_words = [words[idx] for idx in top_indices]
    top_words_articles[jahr] = top_words

# DataFrame aus der Top-Wörter-Tabelle erstellen
top_words_df = pd.DataFrame(top_words_articles)

# Ergebnisse anzeigen
print(top_words_df)

In [ ]:
# Optional: Tabelle speichern
top_words_df.to_csv('wichtige_wörter_artikel_jahr.csv', index=False, sep=';')

### In den Reden

In [ ]:
partei_tfidf_reduced = pd.read_csv("partei_tfidf.csv")

In [ ]:
# Dictionary zur Speicherung der Top-Wörter jeder Partei
top_words_party = {}

# Iteriere über die Parteien
for party in partei_tfidf_reduced['partei'].unique():
    # Filtern der TF-IDF-Werte für die aktuelle Partei
    party_data = partei_tfidf_reduced[partei_tfidf_reduced['partei'] == party].iloc[:, 1:]  # Ohne Partei-Spalte

    # Umwandeln der TF-IDF-Werte und Wörter in eine sortierbare Struktur
    tfidf_values = party_data.values.flatten()
    words = party_data.columns
    top_indices = tfidf_values.argsort()[::-1][:20]  # Indizes der Top 20 höchsten Werte

    # Speichern der Wörter für die Tabelle
    top_words = [words[idx] for idx in top_indices]
    top_words_party[party] = top_words

# DataFrame aus der Top-Wörter-Tabelle erstellen
top_words_party_df = pd.DataFrame(top_words_party)

# Ergebnisse anzeigen
print(top_words_party_df)

In [ ]:
# Exportieren der Tabelle mit Semikolon als Delimiter (für übersichtlicheres Layout)
top_words_party_df.to_csv('wichtige_wörter_reden_partei.csv', index=False, sep=';')